In [1]:
# !pip install --upgrade pip
# !pip install Turicreate

In [2]:
import turicreate as tc

In [3]:
path="AAPL.csv"

In [4]:
# Load the data
data =  tc.SFrame(path)

Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/AAPL.csv

Parsing completed. Parsed 100 lines in 0.042376 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,float,float,float,float,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Unable to interpret "null" as a integer
Parse failed at token ending at: 
	1981-08-10,null,^null,null,null,null,null
Successfully parsed 1 tokens: 
	0: 1981-08-10

1 lines failed to parse correctly

Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/AAPL.csv

Parsing completed. Parsed 10059 lines in 0.016668 secs.

In [5]:
data[1:3]

Date,Open,High,Low,Close,Adj Close,Volume
1980-12-15,0.12221,0.12221,0.121652,0.121652,0.095978,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088934,105728000


In [6]:
# Make a train-test split
train_data, test_data = data.random_split(0.8)

In [7]:

# Automatically picks the right model based on your data.
model = tc.regression.create(
    train_data, 
    target='Close',
    features = [
        'Open',
        'High',
        'Low',
        'Adj Close'
    ])

Automatically generating validation set from 5% of the data.

Linear regression:

--------------------------------------------------------

Number of examples          : 7665

Number of features          : 4

...

| 3         | 0.036541     | 59.128044          | 42.268051            | 7.072547                        | 7.799958                          |

| 4         | 0.044744     | 46.498840          | 29.638847            | 4.996839                        | 5.505967                          |

| 5         | 0.055134     | 36.153397          | 20.624611            | 3.534853                        | 3.858385                          |

| 10        | 0.081244     | 13.128624          | 3.727394             | 0.674188                        | 0.690919                          |

+-----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+

In [24]:
# Save predictions to an SArray
predictions = model.predict(test_data)
# predictions

In [40]:
predictions[1600]

17.942754285226233

In [41]:
test_data[1600]["Close"] # dict

18.074642

In [11]:
# test_data.explore()

In [20]:
# Evaluate the model and save the results into a dictionary
results = model.evaluate( test_data )

In [21]:
results

{'max_error': 3.0828691726718844, 'rmse': 0.16496665753823528}

In [14]:
# Export to Core ML
model.export_coreml('AAPL.mlmodel')